 # Fonction Python et symbolique — Solutions Complètes
 Ce fichier peut être exécuté cellule par cellule dans VSCode.

In [1]:
import openturns as ot
import math
import time
import sys
import numpy as np

 ### Solution de l'exercice 1 : une fonction avec 4 entrées

In [2]:
def simulator(x):
    y1 = x[0] + x[1] + x[2]
    y2 = x[0] - x[1] * x[2]
    y3 = 2 * x[0] + 3 * x[1] + 4 * x[3]
    y = [y1, y2, y3]
    return y

function = ot.PythonFunction(4, 3, simulator)

# Création des lois marginales
X0 = ot.Normal(0.0, 1.0)
X1 = ot.Normal(0.0, 1.0)
X2 = ot.Normal(0.0, 1.0)
X3 = ot.Normal(0.0, 1.0)

# Création de la loi de probabilité conjointe
inputDistribution = ot.JointDistribution((X0, X1, X2, X3))
inputRandomVector = ot.RandomVector(inputDistribution)
outputVariableOfInterest = ot.CompositeRandomVector(function, inputRandomVector)

# Étude probabiliste
sampleSize = 10000
outputSample = outputVariableOfInterest.getSample(sampleSize)
print(f"Moyenne empirique (Exercice 1) : {outputSample.computeMean()}")

Moyenne empirique (Exercice 1) : [-0.0166778,-0.0123527,-0.0277928]


 Une alternative consiste à définir directement un vecteur gaussien en dimension 4.

In [3]:
inputDistribution_alt = ot.Normal(4)

 ### Solution de l'exercice 2 : gradient d'une fonction Python

In [4]:
def simulator_ex2(x):
    y1 = x[0] + x[1] + x[2]
    y2 = x[0] - x[1] * x[2]
    y = [y1, y2]
    return y

function_ex2 = ot.PythonFunction(3, 2, simulator_ex2)
point = [1.0, 2.0, 3.0]

# Gradient par différences finies
d = function_ex2.gradient(point)
print(f"Gradient par différences finies :\n{d}")

# Hessienne
dd = function_ex2.hessian(point)
print(f"Hessienne :\n{dd}")

# Configuration différences finies non centrées
functionImpl = function_ex2.getEvaluation()
myGradient = ot.NonCenteredFiniteDifferenceGradient(1.0e-2, functionImpl)
function_ex2.setGradient(myGradient)
print(f"Gradient (DF non centrées, pas 1e-2) :\n{function_ex2.gradient(point)}")

# Gradient analytique
def simulatorGradient(x):
    dyx1 = [1.0, 1.0]
    dyx2 = [1.0, -x[2]]
    dyx3 = [1.0, -x[1]]
    return [dyx1, dyx2, dyx3]

functionExact = ot.PythonFunction(3, 2, simulator_ex2, gradient=simulatorGradient)
print(f"Gradient analytique :\n{functionExact.gradient(point)}")

Gradient par différences finies :
[[  1  1 ]
 [  1 -3 ]
 [  1 -2 ]]
Hessienne :
sheet #0
[[  0            2.22045e-08  0           ]
 [  2.22045e-08  0            2.22045e-08 ]
 [  0            2.22045e-08  0           ]]
sheet #1
[[  0            0            0           ]
 [  0            0           -1           ]
 [  0           -1            0           ]]
Gradient (DF non centrées, pas 1e-2) :
[[  1  1 ]
 [  1 -3 ]
 [  1 -2 ]]
Gradient analytique :
[[  1  1 ]
 [  1 -3 ]
 [  1 -2 ]]


 ### Solution de l'exercice 3 : gestion de l'historique

In [5]:
function_mem = ot.PythonFunction(3, 2, simulator_ex2)
function_mem = ot.MemoizeFunction(function_mem)

inputRV = ot.RandomVector(ot.Normal(3))
outputRV = ot.CompositeRandomVector(function_mem, inputRV)

outputSample_mem = outputRV.getSample(20)

inputs_hist = function_mem.getInputHistory()
outputs_hist = function_mem.getOutputHistory()
print(f"Historique avant nettoyage : {inputs_hist.getSize()} éléments.")

function_mem.clearHistory()
print(f"Historique après nettoyage : {function_mem.getOutputHistory().getSize()} éléments.")

Historique avant nettoyage : 20 éléments.
Historique après nettoyage : 0 éléments.


 ### Solution de l'exercice 4 : fonction symbolique avec 4 entrées

In [6]:
myfunctionSymbolic4 = ot.SymbolicFunction(
    ("x1", "x2", "x3", "x4"),
    ("x1 + x2 + x3", "x1 - x2 * x3", "2 * x1 + 3 * x2 + 4 * x4"),
)
X_val = ot.Point([1.0, 2.0, 3.0, 4.0])
print(f"Évaluation symbolique au point X : {myfunctionSymbolic4(X_val)}")

Évaluation symbolique au point X : [6,-5,24]


 ### Solution de l'exercice 5 : fonction symbolique avec paramètres

In [7]:
full_func = ot.SymbolicFunction(
    ("x1", "x2", "a", "b", "c", "d"), ("a * x1 + b * x2", "c * x1 + d * x2")
)

a, b, c, d = 12.0, 23.0, -34.0, 45.0
indices = [2, 3, 4, 5]
referencePoint = [a, b, c, d]
g_param = ot.ParametricFunction(full_func, indices, referencePoint)

print(f"Évaluation paramétrique (x1=1, x2=2) : {g_param([1.0, 2.0])}")

Évaluation paramétrique (x1=1, x2=2) : [58,56]


 ### Solution de l'exercice 6 : gradient d'une fonction symbolique

In [8]:
sym_func = ot.SymbolicFunction(("x1", "x2", "x3"), ("x1 + x2 + x3", "x1 - x2 * x3"))
grad_op = sym_func.getGradient()
print("Opérateur de gradient symbolique :")
print(grad_op)
print(f"Valeur au point (1,2,3) :\n{grad_op.gradient([1.0, 2.0, 3.0])}")

Opérateur de gradient symbolique :

| d(y0) / d(x1) = 1
| d(y0) / d(x2) = 1
| d(y0) / d(x3) = 1
| d(y1) / d(x1) = 1
| d(y1) / d(x2) = -1*x3
| d(y1) / d(x3) = -1*x2

Valeur au point (1,2,3) :
[[  1  1 ]
 [  1 -3 ]
 [  1 -2 ]]


 ### Solution de l'exercice 7 : variables intermédiaires

In [9]:
inputs_inter = ["X1", "X2"]
outputs_inter = ["Y1", "Y2"]
formula_inter = "var S := X1 + X2; var T := X1 * X2; Y1 := S + T; Y2 := S * T"
myFunction_inter = ot.SymbolicFunction(inputs_inter, outputs_inter, formula_inter)
print(f"Résultat variables intermédiaires : {myFunction_inter([1.0, 2.0])}")

Résultat variables intermédiaires : [5,6]


 ### Solution de l'exercice 8 : performance et parallélisation

In [10]:
def highDimensionSimulator(x):
    xPoint = ot.Point(x)
    dim_val = xPoint.getDimension()
    y0, y1 = 0.0, 1.0
    for i in range(dim_val):
        y0 += math.exp(x[i])
        y1 *= math.exp(x[i])
    return [y0, y1]

dim_h = 100
inputRV_h = ot.RandomVector(ot.Normal(dim_h))
sampleSize_h = 100

# Test Séquentiel
f_seq = ot.PythonFunction(dim_h, 2, highDimensionSimulator)
t0 = time.time()
ot.CompositeRandomVector(f_seq, inputRV_h).getSample(sampleSize_h)
print(f"Temps séquentiel : {time.time() - t0:.2f} s")

# Test Parallèle
if not sys.platform.startswith("win"):
    f_par = ot.PythonFunction(dim_h, 2, highDimensionSimulator, n_cpus=8)
    t0 = time.time()
    ot.CompositeRandomVector(f_par, inputRV_h).getSample(sampleSize_h)
    print(f"Temps parallèle : {time.time() - t0:.2f} s")
else:
    print("Parallélisation n_cpus ignorée (Windows).")

Temps séquentiel : 0.01 s
Parallélisation n_cpus ignorée (Windows).
